In [0]:
import pandas as pd

import requests
import json
import os
import datetime
import time

import pyspark.sql.functions as F

You will need to create a notebook with your keys in it in a notebook called `keys`. The next two lines provide examples of how to reference that notebook.

In [0]:
%run ./keys

In [0]:
%run ../sp23/keys

ow_key is the variable
d8a


In [0]:
# https://openweathermap.org/forecast16
api_key = ow_key
# https://api.openweathermap.org/data/2.5/forecast?lat=43.825386&lon=-111.792824&appid=d8aa64c57714e98a56057fa3bd9f478a&units=imperial&cnt=5
lat = "43.825386"
lon = "-111.792824"
url = "https://api.openweathermap.org/data/2.5/forecast?lat=%s&lon=%s&appid=%s&units=imperial&cnt=5" % (lat, lon, api_key)
print(url[0:85] + "<YOURKEY>" + url[117:])


https://api.openweathermap.org/data/2.5/forecast?lat=43.825386&lon=-111.792824&appid=<YOURKEY>&units=imperial&cnt=5


ow_key is the variable
d8a


In [0]:
# 200 is a good sign, #400 is a bad sign
response = requests.get(url)
response

Out[51]: <Response [200]>

In [0]:
data = json.loads(response.text)
data

Out[52]: {'cod': '200',
 'message': 0,
 'cnt': 5,
 'list': [{'dt': 1685394000,
   'main': {'temp': 72.14,
    'feels_like': 70.7,
    'temp_min': 69.12,
    'temp_max': 72.14,
    'pressure': 1015,
    'sea_level': 1015,
    'grnd_level': 852,
    'humidity': 35,
    'temp_kf': 1.68},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 11.14, 'deg': 217, 'gust': 14.92},
   'visibility': 10000,
   'pop': 0.31,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-05-29 21:00:00'},
  {'dt': 1685404800,
   'main': {'temp': 71.44,
    'feels_like': 70.07,
    'temp_min': 70.03,
    'temp_max': 71.44,
    'pressure': 1012,
    'sea_level': 1012,
    'grnd_level': 850,
    'humidity': 38,
    'temp_kf': 0.78},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 18},
   'wind': {'speed': 10.94, 'deg': 213, 'gust': 14.47},
   'visibil

In [0]:
# Google: python print json.loads object -> https://www.digitalocean.com/community/tutorials/python-pretty-print-json
json_formatted_str = json.dumps(data, indent=3)
print(json_formatted_str)

{
   "cod": "200",
   "message": 0,
   "cnt": 5,
   "list": [
      {
         "dt": 1685394000,
         "main": {
            "temp": 72.14,
            "feels_like": 70.7,
            "temp_min": 69.12,
            "temp_max": 72.14,
            "pressure": 1015,
            "sea_level": 1015,
            "grnd_level": 852,
            "humidity": 35,
            "temp_kf": 1.68
         },
         "weather": [
            {
               "id": 800,
               "main": "Clear",
               "description": "clear sky",
               "icon": "01d"
            }
         ],
         "clouds": {
            "all": 0
         },
         "wind": {
            "speed": 11.14,
            "deg": 217,
            "gust": 14.92
         },
         "visibility": 10000,
         "pop": 0.31,
         "sys": {
            "pod": "d"
         },
         "dt_txt": "2023-05-29 21:00:00"
      },
      {
         "dt": 1685404800,
         "main": {
            "temp": 71.44,
            

In [0]:
rdd = spark.sparkContext.parallelize([response.text])
df = spark.read.json(rdd)
display(df)

city,cnt,cod,list,message
"List(List(43.8254, -111.7928), US, 5605242, Rexburg, 25484, 1685360996, 1685415573, -21600)",5,200,"List(List(List(0), 1685394000, 2023-05-29 21:00:00, List(70.7, 852, 35, 1015, 1015, 72.14, 1.68, 72.14, 69.12), 0.31, null, List(d), 10000, List(List(clear sky, 01d, 800, Clear)), List(217, 14.92, 11.14)), List(List(18), 1685404800, 2023-05-30 00:00:00, List(70.07, 850, 38, 1012, 1012, 71.44, 0.78, 71.44, 70.03), 0.34, null, List(d), 10000, List(List(few clouds, 02d, 801, Clouds)), List(213, 14.47, 10.94)), List(List(56), 1685415600, 2023-05-30 03:00:00, List(60.89, 850, 59, 1012, 1012, 62.2, 2.76, 62.2, 57.24), 0.71, List(0.13), List(n), 10000, List(List(light rain, 10n, 500, Rain)), List(209, 17.58, 8.81)), List(List(68), 1685426400, 2023-05-30 06:00:00, List(48.79, 849, 84, 1012, 1012, 50.13, 0.0, 50.13, 50.13), 0.42, null, List(n), 10000, List(List(broken clouds, 04n, 803, Clouds)), List(135, 10.76, 7.38)), List(List(99), 1685437200, 2023-05-30 09:00:00, List(46.87, 848, 82, 1012, 1012, 48.56, 0.0, 48.56, 48.56), 0.05, null, List(n), 10000, List(List(overcast clouds, 04n, 804, Clouds)), List(71, 4.76, 4.47)))",0


In [0]:
df.printSchema()

root
 |-- city: struct (nullable = true)
 |    |-- coord: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- population: long (nullable = true)
 |    |-- sunrise: long (nullable = true)
 |    |-- sunset: long (nullable = true)
 |    |-- timezone: long (nullable = true)
 |-- cnt: long (nullable = true)
 |-- cod: string (nullable = true)
 |-- list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- clouds: struct (nullable = true)
 |    |    |    |-- all: long (nullable = true)
 |    |    |-- dt: long (nullable = true)
 |    |    |-- dt_txt: string (nullable = true)
 |    |    |-- main: struct (nullable = true)
 |    |    |    |-- feels_like: double (nullable = true)
 |    |    |    |-- grnd_level: long (nullable = true)
 |    |    |    |-- humidity: long (nul